In [ ]:
# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
def int64_feature(values):
    if not isinstance(values, (tuple, list)):
        values = [values]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

def bytes_feature(values):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))

def image_to_tfexample(image_data, image_format, height, width, class_id):
    """
        Creates a TFRecord example image entry.
    Args:
        image_data: The image as numpy array
        image_format: The image format type e.g. jpg
        height and width: The image sizes
        class_id: The image label
    """
    return tf.train.Example(features=tf.train.Features(feature={
      'image/encoded': bytes_feature(image_data),
      'image/format': bytes_feature(image_format),
      'image/class/label': int64_feature(class_id),
      'image/height': int64_feature(height),
      'image/width': int64_feature(width),
  }))

In [ ]:
def write_tfrecord(images_by_id, top_dir, output_filename, image_height, image_width):
    """
        Creates a TFRecord file from a map of image files paths to class ids
    Args:
        images_by_id: Map of image file paths mapped to class id
        top_dir: where to put the file
        output_filename: how to name the file
    """
    tf.reset_default_graph()
    
    image_path = tf.placeholder(dtype=tf.string)
    image_raw = tf.read_file(image_path)
    
    errors = []
    num_images = sum([len(v) for _, v in images_by_id.items()])
    with tf.Session() as sess:
        with tf.python_io.TFRecordWriter("/".join([top_dir, output_filename])) as tfrecord_writer:
            processed_count = 0
            for class_id, images in images_by_id.items():
                for image in images:
                    # Show progress
                    processed_count += 1
                    print('>> Converting image %d/%d' % (processed_count, num_images), end="\r")
                    try:
                        image_data = sess.run(image_raw, feed_dict={image_path: image})
                        example = image_to_tfexample(
                            image_data,
                            b'jpg',
                            image_height,
                            image_width,
                            class_id)
                        tfrecord_writer.write(example.SerializeToString())
                    except:
                        err_msg = sys.exc_info()[0]
                        errors.append((image, err_msg))
                
    print()            
    print("Errors: {}".format(len(errors)))
    if len(errors) > 0:
        for (error_file, info) in errors:
            print("Error one file: {} because: {}".format(error_file, info))

In [ ]:
def read_sample(example_raw):
    """
        Read a single TFRecord example and converting into an image
    Args:
        The TFRecord example that represents an image
    """
    example = tf.parse_single_example(
        example_raw,
        features={
            'image/encoded': tf.FixedLenFeature([], tf.string),
            'image/format': tf.FixedLenFeature([], tf.string),
            'image/class/label': tf.FixedLenFeature([], tf.int64),
            'image/height': tf.FixedLenFeature([], tf.int64),
            'image/width': tf.FixedLenFeature([], tf.int64),
        })
    image = tf.image.decode_image(example['image/encoded'])
    height = tf.cast(example['image/height'], tf.int32)
    width = tf.cast(example['image/width'], tf.int32)  
    label = example["image/class/label"]
    image = tf.reshape(image, (height, width))
    return image, label

def make_dataset(tfrecord_filepath):
    """
        Returns a dataset ready to process a TFRecord file
    """
    dataset = tf.data.TFRecordDataset(tfrecord_filepath)
    dataset = dataset.map(read_sample)
    return dataset 

In [ ]:
# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
# Create mapping files for class ids to class names
def write_label_file(labels_to_class_names, dataset_dir,
                     filename=LABELS_FILENAME):
    """
        Writes a file with the list of class names row-wise like "1:class1".
    Args:
        labels_to_class_names: A map of (integer) labels to class names.
        dataset_dir: The directory in which the labels file should be written.
        filename: The filename where the class names are written.
    """
    labels_filename = os.path.join(dataset_dir, filename)
    with tf.gfile.Open(labels_filename, 'w') as f:
        for label in labels_to_class_names:
            class_name = labels_to_class_names[label]
            f.write('%d:%s\n' % (label, class_name))
            
def read_label_file(dataset_dir, filename=LABELS_FILENAME):
    """Reads the labels file and returns a mapping from ID to class name.
    Args:
        dataset_dir: The directory in which the labels file is found.
        filename: The filename where the class names are written.
    Returns:
        A map from a label (integer) to class name.
    """
    labels_filename = os.path.join(dataset_dir, filename)
    with tf.gfile.Open(labels_filename, 'rb') as f:
        lines = f.read().decode()
        lines = lines.split('\n')
        lines = filter(None, lines)

    labels_to_class_names = {}
    for line in lines:
        index = line.index(':')
        labels_to_class_names[int(line[:index])] = line[index+1:]
    return labels_to_class_names